# Delta Hedging

### *Environment Setup*

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import calendar
import holidays
from datetime import date, timedelta
from tqdm import tqdm
tqdm.pandas() 

import sys
sys.path.append('../')

In [2]:
options_data = pd.read_csv('../data/SPYOPT.csv',index_col=0, parse_dates=True)
futures_data = pd.read_csv('../data/SPYFUT.csv',index_col=0, parse_dates=True)
risk_free_rate = pd.read_csv('../data/RFRate.csv', index_col=0, parse_dates=True)

In [3]:
unique_options = options_data['symbol'].unique()
target_options = [option for option in unique_options if not option.startswith('UD:1V')]

In [4]:
options_data = options_data[options_data['symbol'].isin(target_options)].drop(columns=['rtype','publisher_id','instrument_id'])

In [5]:
exp_months = {'F': 1, 'G': 2, 'H': 3, 'J': 4, 'K': 5, 'M': 6, 'N': 7, 'Q': 8, 'U': 9, 'V': 10, 'X': 11, 'Z': 12}
exp_years = {'0': 2020, '1': 2021, '2': 2022, '3': 2023, '4': 2024, '5': 2025, '6': 2026,'7':2027, '9':2019,'8':2018}

In [6]:
def get_thursday (exp_month,exp_year):
    last_day = calendar.monthrange(exp_year, exp_month)[1]
    last_date = date(exp_year, exp_month, last_day)
    
    while last_date.weekday() != 3:
        last_date -= timedelta(days=1)
    
    return last_date

def get_expiration_date(exp_month, exp_year):
    holiday_list = holidays.CountryHoliday('US')
    expiry = get_thursday(exp_month,exp_year)

    while expiry in holiday_list:
        expiry -= timedelta(days=1)
    
    return expiry

In [7]:
def grouping_function(x):
    symbol = x['symbol'].iloc[0]
    exp_date , strike_type = symbol.split(' ') 

    exp_month = exp_months[exp_date[-2]]
    exp_year = exp_years[exp_date[-1]]
    expiry = get_expiration_date(exp_month, exp_year)

    type = strike_type[0]
    strike = float(strike_type[1:])

    midprice = (x['high'] + x['low']) / 2

    x['Maturity'] = expiry.strftime('%Y-%m-%d')
    x['Type'] = type
    x['Strike'] = strike
    x['MidPrice'] = midprice

    x = x.drop(columns=['high', 'low', 'open', 'close'])

    return x

def time_to_expiry(ts_event, maturity):
    ts_event = pd.to_datetime(ts_event)
    maturity = pd.to_datetime(maturity)
    diff = (maturity - ts_event).dt.days / 365.25
    diff = diff.where(diff >= 0, diff + 10)
    return diff

In [8]:
grouped_options_data = options_data.groupby('symbol').apply(grouping_function).drop(columns=['symbol']).reset_index().drop(columns=['symbol'])
grouped_options_data = grouped_options_data[['ts_event', 'Maturity','Strike', 'MidPrice', 'Type', ]]

/var/folders/7k/ytnljb093xg6bbfksvnhyg940000gn/T/ipykernel_57904/1341014864.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_options_data = options_data.groupby('symbol').apply(grouping_function).drop(columns=['symbol']).reset_index().drop(columns=['symbol'])


In [9]:
grouped_options_data['T'] = time_to_expiry(grouped_options_data['ts_event'], grouped_options_data['Maturity'])

### Preparing Futures Data

In [10]:
def futures_grouper(x):
    symbol = x['symbol'].iloc[0]
    exp_month = exp_months[symbol[-2]]
    exp_year = exp_years[symbol[-1]]
    expiry = get_expiration_date(exp_month, exp_year)

    x['Maturity'] = expiry.strftime('%Y-%m-%d')
    x['MidPrice'] = (x['high'] + x['low']) / 2
    x = x.drop(columns=['high', 'low', 'open', 'close'])

    return x

In [11]:
filtered_futures_data = futures_data[futures_data['symbol'].str.len() == 4]
grouped_futures_data = filtered_futures_data.groupby('symbol').apply(futures_grouper).drop(columns=['symbol']).reset_index().drop(columns=['rtype', 'publisher_id', 'instrument_id', 'symbol','volume'])

/var/folders/7k/ytnljb093xg6bbfksvnhyg940000gn/T/ipykernel_57904/3610527438.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_futures_data = filtered_futures_data.groupby('symbol').apply(futures_grouper).drop(columns=['symbol']).reset_index().drop(columns=['rtype', 'publisher_id', 'instrument_id', 'symbol','volume'])


In [12]:
df_options_indexed = grouped_options_data.set_index(['ts_event', 'Maturity'])
df_futures_indexed = grouped_futures_data.set_index(['ts_event', 'Maturity'])
df_futures_indexed = df_futures_indexed.rename(columns={'MidPrice': 'FuturesPrice'})
df_joined = df_options_indexed.join(df_futures_indexed, how='left')
df_joined = df_joined.reset_index().set_index(['ts_event'])
df_joined = df_joined.sort_index()

In [13]:
risk_free_rate_full_range = pd.date_range(start=risk_free_rate.index.min(), end=risk_free_rate.index.max(), freq='D')
risk_free_rate_full = risk_free_rate.reindex(risk_free_rate_full_range)
risk_free_rate_full = risk_free_rate_full.ffill()

In [15]:
risk_free_rate_indexed = risk_free_rate_full.reset_index()
risk_free_rate_indexed.columns = ['ts_event', 'RiskFreeRate']
risk_free_rate_indexed['ts_event'] = pd.to_datetime(risk_free_rate_indexed['ts_event'])
df_joined = df_joined.merge(risk_free_rate_indexed,on='ts_event', how='left')

In [16]:
df_joined

,ts_event,Maturity,Strike,MidPrice,Type,T,FuturesPrice,RiskFreeRate
0,2018-05-13,2018-06-28,2680.0,17.500,P,0.125941,2731.875,1.68
1,2018-05-13,2018-06-28,2505.0,3.500,P,0.125941,2731.875,1.68
2,2018-05-13,2018-06-28,2730.0,31.500,P,0.125941,2731.875,1.68
3,2018-05-13,2018-06-28,2250.0,0.925,P,0.125941,2731.875,1.68
4,2018-05-13,2018-06-28,2800.0,8.375,C,0.125941,2731.875,1.68
...,...,...,...,...,...,...,...,...
841095,2025-06-26,2025-12-24,5250.0,71.000,P,0.495551,6226.000,4.11
841096,2025-06-26,2025-09-25,5875.0,91.000,P,0.249144,6174.875,4.11
841097,2025-06-26,2025-09-25,6250.0,200.625,P,0.249144,6174.875,4.11
841098,2025-06-26,2026-03-26,1800.0,3.950,P,0.747433,6283.875,4.11


In [17]:
from scipy.optimize import brentq
from scipy.stats import norm
import numpy as np

def bs_price(S, K, T, r, sigma, option_type='C'):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'C':
        return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def implied_vol(mid_price, S, K, T, r, option_type='C'):
    try:
        return brentq(lambda sigma: bs_price(S, K, T, r, sigma, option_type) - mid_price, 1e-6, 5.0)
    except:
        return np.nan


In [18]:
df_joined['IV'] = df_joined.progress_apply(lambda row: implied_vol( mid_price=row['MidPrice'],
                                                        S=row['FuturesPrice'],
                                                        K=row['Strike'],
                                                        T=row['T'],
                                                        r = row['RiskFreeRate'],
                                                        option_type=row['Type']),
                                                        axis=1)
                                                        

100%|██████████| 841100/841100 [07:33<00:00, 1854.03it/s]


In [20]:
df_joined.to_csv('../data/JoinedData.csv')